# Qwen3-235B-A22B-Instruct-2507 

模型架构与技术亮点
Qwen3-235B-A22B-Instruct-2507 采用稀疏专家（Mixture of Experts）架构，具有以下关键特性：
* 参数结构：总参数规模达 2350 亿，但每次推理仅激活其中的 220 亿参数（8 个专家，来自 128 个候选专家），大幅优化推理效率；
* 层数与注意力机制：模型拥有 94 层 Transformer，采用 GQA（Grouped Query Attention）机制，进一步增强推理稳定性；
* 超长上下文支持：原生支持 256K token 长度，具备极强的长文本处理能力；
* 多种部署形式：官方同步发布了 FP8 精度版本，支持更低成本的本地部署，同时兼容 Transformer、vLLM、SGLang 等主流推理框架。

Grouped Query Attention（GQA）即分组查询注意力，是对 Multi-Head Attention（MHA）和 Multi-Query Attention（MQA）的扩展，旨在平衡标准多头注意力和多查询注意力之间的权衡，在计算效率和模型表达能力之间提供灵活的权衡。具体介绍如下：
* 核心原理：GQA 将查询头分成了 G 个组，每个组共享一个公共的键（K）和值（V）投影。通过这种方式，它可以减少存储每个头的键和值所需的内存开销，特别是在处理大的上下文窗口或批次大小时效果更为明显。
* 主要变体：GQA 有三种变体。GQA-1 属于一个单独的组，这种情况等同于 MQA；GQA-H 的组数等于头数，基本与 MHA 相同；GQA-G 是一个中间配置，具有 G 个组，能够较好地平衡效率和表达能力。
* 实现方式：在最简实现形式中，可通过定义 GroupedQueryAttention 类来实现，该类继承自 Attention 类并重写 forward 方法。具体过程是先将查询头分组，然后对每一个查询组分别计算注意力分数和输出，最后将所有查询组的输出拼接起来，再与投影矩阵进行矩阵乘法运算，得到 GQA 的最终输出。

独立 Instruct 微调策略
本次发布的模型采用了全新的训练策略：完全独立的 Instruct 分支，不再混合使用“思考模式”与“指令模式”。
这种分离式训练使模型更专注于执行用户明确指令，在文本生成、问题解答、工具调用等任务中表现更加稳定可靠。此外，模型默认关闭“<think>”风格的思考生成，使其在对话系统中响应更简洁、高效，更适合实时交互和服务部署。
* https://juejin.cn/post/7507869468400582690 混合模式
* 训练数据构造：构造一个混合思考的 system template。如果训练数据需要思考，那么在 user query 后面加上 “/think” 表示符号，然后模型需要填充 “{thinking_content}” 内容；而如果不需要思考，则在 user query 后加上 “/no_think”，让模型思考标签中的内容变成 “<think></think>”，以此保证非思考数据的多样性。通过这种方式构造出训练数据，使模型初步具备混合思考的能力。
* 强化学习：在监督微调（SFT）之后，为了强化模型对于 “/think” 或者 “/no_think” 指令的遵循，在强化学习阶段加入 “format - following” 的格式奖励。当 user query 中有 “/think” 时，模型的回复需要是 “<think> a lot of thinking</think> final respone”；当 user query 中有 “/no_think” 时，模型的回复需要是 “<think></think> final respone”。
* 推理阶段：用户可以手动在 prompt 中写上 “/think” 或者 “/no_think” 来启动或者终止思考。也可以通过设置参数来控制，当模型不加 “enable_thinking = False” 的时候，模型默认思考，加了之后默认不思考。在多轮对话中，模型会遵循最近的指令。此外，还可以根据用户设置的 max_tokens 手动终止思考过程，当快接近 max_tokens 的时候，拼接特定语句让模型接着生成最终回复。


HF：https://huggingface.co/Qwen/Qwen3-235B-A22B-Instruct-2507
